In [7]:
# Load all the recordings in the data path to a list
import os
data_path = '/workspaces/RovVision2_Sonar/bags'
output_path = '/workspaces/RovVision2_Sonar/bags/output'
# If there isn't a subfolder called 'output', create it
if not os.path.exists(output_path):
    os.makedirs(output_path)

recordings = []

# Get all folders in the data path, excluding the 'output_path' folder
for folder in os.listdir(data_path):
    if folder == 'output':
        continue
    
    # Check if the folder is a directory
    if os.path.isdir(os.path.join(data_path, folder)):
        # Append the folder name to the recordings list
        recordings.append(folder)

# Print the recordings list
print(recordings)


['20221211_092506']


In [8]:
# Create an output folder for each recording
for record in recordings:
    print(f"Processing {record}")
    # Create the output folder for the current recording
    output_folder = os.path.join(output_path, record)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)



Processing 20221211_092506


In [9]:
import pandas
import datetime
# For each recording, load the csv file
for record in recordings:
    print(f"Processing {record}")
    # Get the path to the csv file
    csv_file = os.path.join(data_path, record, 'data.csv')
    # Open the csv file
    data = pandas.read_csv(csv_file)
    # Print the data
    print(data.head())

    # Change all 
    # 'topic_depth' values to 'depth',
    # 'topic_stereo_camera' values to 'camera',
    # 'topic_sonar' values to 'sonar'
    data['topic'] = data['topic'].replace('topic_depth', 'depth')
    data['topic'] = data['topic'].replace('topic_stereo_camera', 'camera')
    data['topic'] = data['topic'].replace('topic_sonar', 'sonar') 

    # For each modality, save the data to a separate subfolder named after the modality and create a csv file
    for modality in data['topic'].unique():
        print(f"Processing {modality}")
        # Get the data for the current modality
        modality_data = data[data['topic'] == modality]
        # Get the output folder for the current recording
        output_folder = os.path.join(output_path, record)
        # Create the output folder for the current modality
        modality_folder = os.path.join(output_folder, modality)
        if not os.path.exists(modality_folder):
            os.makedirs(modality_folder)

        # Create a new df with the 'timestamp' and 'filename' columns
        new_data = pandas.DataFrame(columns=['timestamp', 'filename'])
        
        
        # For the 'sonar' and 'camera' modalities, copy the images to the output folder and create a csv file with 'timestamp' and 'filename' columns,
        # the new filename should be the modality idx.
        modality_idx = 1
        if modality == 'sonar' or modality == 'camera':
            for index, row in modality_data.iterrows():
                # Get the image path
                print(row)
                # break
                filename = f"{int(row['value']):08d}"
                image_path = os.path.join(data_path, record, 'imgs', filename + '.png')
                # Get the new image path
                new_image_path = os.path.join(modality_folder, f"{modality_idx}.png")
                # Copy the image to the new image path
                os.system(f"cp {image_path} {new_image_path}")
                # Append the 'timestamp' and 'filename' to the new_data df
                # date_value = row['date']
                # time_value = row['time']
                # timestamp_from_date_and_time = datetime.datetime.strptime(date_value + ' ' + time_value, '%Y_%m_%d %H_%M_%S_%f')
                # timestamp = timestamp_from_date_and_time.timestamp()
                timestamp = row['timestamp']
                new_data = new_data.append({'timestamp': timestamp, 'filename': f"{modality_idx:08d}.png"}, ignore_index=True)
                # Increment the modality index
                modality_idx += 1    

            # Save the modality data to a csv file
            new_data.to_csv(os.path.join(modality_folder, modality + '.csv'), index=False)
        else:
            # For the 'depth' modality, create a csv file with 'timestamp' and 'value' columns
            new_data = pandas.DataFrame(columns=['timestamp', 'value'])
            for index, row in modality_data.iterrows():
                # Append the 'timestamp' and 'value' to the new_data df
                timestamp = row['timestamp']
                new_data = new_data.append({'timestamp': timestamp, 'value': row['value']}, ignore_index=True)
            # Save the modality data to a csv file
            new_data.to_csv(os.path.join(modality_folder, modality + '.csv'), index=False)
        

        

Processing 20221211_092506
                 topic          bagfile                       value  \
0  topic_stereo_camera  20221211_092506  2022_12_11_07_25_06_299462   
1          topic_depth  20221211_092506                        0.14   
2          topic_sonar  20221211_092506  2022_12_11_07_25_06_788783   
3  topic_stereo_camera  20221211_092506  2022_12_11_07_25_06_794995   
4          topic_depth  20221211_092506                       0.145   

         date             time     timestamp  
0  2022_12_11  07_25_06_299462  1.670744e+09  
1  2022_12_11  07_25_06_788783  1.670744e+09  
2  2022_12_11  07_25_06_788783  1.670744e+09  
3  2022_12_11  07_25_06_794995  1.670744e+09  
4  2022_12_11  07_25_06_794995  1.670744e+09  
Processing camera
topic                            camera
bagfile                 20221211_092506
value        2022_12_11_07_25_06_299462
date                         2022_12_11
time                    07_25_06_299462
timestamp                   1.67074e+09
Name: 

In [10]:
import pandas as pd
import os

def find_closest_match(timestamp, data):
    min_diff = 0.05
    closest_match = None

    for index, data_timestamp in enumerate(data, start=1):
        diff = abs(timestamp - data_timestamp)
        if diff < min_diff:
            min_diff = diff
            closest_match = (index, data_timestamp)

    return closest_match

# Load the 'depth' csv file
depth_data = pandas.read_csv(os.path.join(output_path, recordings[0], 'depth', 'depth.csv'))
# Load the 'camera' csv file
camera_data = pandas.read_csv(os.path.join(output_path, recordings[0], 'camera', 'camera.csv'))
# Load the 'sonar' csv file
sonar_data = pandas.read_csv(os.path.join(output_path, recordings[0], 'sonar', 'sonar.csv'))

# Create a new df with the 'timestamp', 'depth', 'camera' and 'sonar' columns
new_data = pandas.DataFrame(columns=['timestamp', 'depth', 'camera', 'sonar'])

# Find matches between modalities
matches = []
# used_indexes = set()
used_camera_indexes = set()
used_sonar_indexes = set()
used_nav_indexes = set()

for camera_index, camera_timestamp in enumerate(camera_data['timestamp'], start=1):
    # Find the closest match in the 'depth' csv file
    depth_match = find_closest_match(camera_timestamp, depth_data['timestamp'])
    # Find the closest match in the 'sonar' csv file
    sonar_match = find_closest_match(camera_timestamp, sonar_data['timestamp'])
    # Find the closest match in the 'nav' csv file
    # nav_match = find_closest_match(camera_timestamp, nav_data['timestamp'])

    if depth_match and sonar_match:
        # Check if the 'depth' and 'sonar' timestamps are not already used
        if depth_match[0] not in used_camera_indexes and sonar_match[0] not in used_camera_indexes:
            # Append the 'depth', 'camera' and 'sonar' values to the new_data df
            new_data = new_data.append({'timestamp': camera_timestamp, 'depth': depth_data['value'][depth_match[0] - 1], 'camera': camera_data['filename'][camera_index - 1], 'sonar': sonar_data['filename'][sonar_match[0] - 1]}, ignore_index=True)
            # Add the 'depth' and 'sonar' indexes to the used_indexes set
            used_camera_indexes.add(camera_index)
            used_camera_indexes.add(depth_match[0])
            used_camera_indexes.add(sonar_match[0])
            # Append the 'depth', 'camera' and 'sonar' timestamps to the matches list
            matches.append((camera_timestamp, depth_data['timestamp'][depth_match[0] - 1], sonar_data['timestamp'][sonar_match[0] - 1]))

# Save the new_data df to a csv file
new_data.to_csv(os.path.join(output_path, recordings[0], 'matches.csv'), index=False)